In [8]:
from datetime import datetime
import random
import numpy as np 
import networkx as nx 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.sparse import csr_matrix
from numpy.linalg import norm
from collections import Counter
import matplotlib.pyplot as plt
from numpy.linalg import norm, eigvalsh

def sbm_gender_homophily_adj_and_metadata(n_g1, n_g2, p_in, p_out, seed):
    np.random.seed(seed)
    sizes = [n_g1, n_g2]
    probs = [[p_in, p_out], [p_out, p_in]]
    G = nx.stochastic_block_model(sizes, probs, seed=seed)
    adj_matrix = nx.to_numpy_array(G)
    metadata = np.zeros((n_g1 + n_g2, 7))
    metadata[:n_g1, 1] = 1  # First block: gender 1
    metadata[n_g1:, 1] = 2  # Second block: gender 2
    return adj_matrix, metadata

def sbm_gender_homophily_adj_and_metadata(n_g1, n_g2, p_in, p_out, seed):
    np.random.seed(seed)
    sizes = [n_g1, n_g2]
    probs = [[p_in, p_out], [p_out, p_in]]
    G = nx.stochastic_block_model(sizes, probs, seed=seed)
    adj_matrix = nx.to_numpy_array(G)
    metadata = np.zeros((n_g1 + n_g2, 7))
    metadata[:n_g1, 1] = 1  # First block: gender 1
    metadata[n_g1:, 1] = 2  # Second block: gender 2
    return adj_matrix, metadata

def create_iid_core_fringe_graph(adj_matrix, k, seed=None, ff=False):
    """
    Creates a core-fringe graph using an IID sample of size k as the core.
    The fringe consists of all nodes connected to the core.
    
    Parameters:
      - adj_matrix (scipy.sparse matrix): full graph adjacency matrix.
      - k (int): number of core nodes to sample.
      - seed (int, optional): random seed.
      
    Returns:
      (core_fringe_adj, core_indices, fringe_indices)
    """
    if seed is not None:
        np.random.seed(seed)
    
    core_indices = np.random.choice(adj_matrix.shape[0], size=k, replace=False)
    is_core = np.zeros(adj_matrix.shape[0], dtype=bool)
    is_core[core_indices] = True

    A = csr_matrix(adj_matrix)
    total_edges_original = int(A.nnz / 2)
    print(f"Total edges in original adjacency: {total_edges_original}")
    neighbors = A[core_indices].nonzero()[1]
    fringe_indices = np.setdiff1d(np.unique(neighbors), core_indices)
    
    fringe_frignge_adj = A[fringe_indices, :][:, fringe_indices]
    # Build a mask that keeps only core-core and core-fringe edges.
    # Create sparse mask matrix
    mask_data = []
    mask_rows = []
    mask_cols = []
    
    # Add core-core edges
    core_core_edges = A[core_indices][:, core_indices]
    for i, j in zip(*np.triu_indices_from(core_core_edges, k=1)):
        mask_rows.extend([core_indices[i], core_indices[j]])
        mask_cols.extend([core_indices[j], core_indices[i]])
        mask_data.extend([1, 1])
    
    # Add core-fringe edges
    core_fringe_edges = A[core_indices][:, fringe_indices]
    for i, j in zip(core_fringe_edges.nonzero()[0], core_fringe_edges.nonzero()[1]):
        mask_rows.extend([core_indices[i], fringe_indices[j]])
        mask_cols.extend([fringe_indices[j], core_indices[i]])
        mask_data.extend([1, 1])

    # Create sparse mask matrix
    mask = csr_matrix((mask_data, (mask_rows, mask_cols)), shape=A.shape)

    core_fringe_adj = A.multiply(mask)

    print(f"IID core")
    print(f"Core size: {len(core_indices)}")
    core_core_edges = int(np.sum(core_fringe_adj[core_indices][:, core_indices]) / 2)
    core_fringe_edges = int(np.sum(core_fringe_adj[core_indices][:, fringe_indices]) / 2)
    print(f"Number of core-core edges: {core_core_edges}")
    print(f"Number of core-fringe edges: {core_fringe_edges}")
    
    # Calculate number of fringe-fringe edges lost
    fringe_fringe_edges = total_edges_original - (core_core_edges + core_fringe_edges)
    print(f"Number of fringe-fringe edges (lost): {fringe_fringe_edges}")

    # After constructing core_fringe_adj
    fringe_adj = core_fringe_adj[fringe_indices, :][:, fringe_indices]
    assert fringe_adj.nnz == 0, "Fringe-fringe edges exist in the core-fringe adjacency matrix!"

    if ff:
        return core_fringe_adj, core_indices, fringe_indices, fringe_frignge_adj
    else:
        return core_fringe_adj, core_indices, fringe_indices

In [9]:
def extract_feature(A, u, v, method='eigenValue2'):
    subA = np.zeros(A.shape)
    subA[u, :] = A[u, :]  # Entire row u
    subA[v, :] = A[v, :]  # Entire row v
    subA[:, u] = A[:, u]  # Entire column u
    subA[:, v] = A[:, v]  # Entire column v
    # Laplacian
    L = np.diag(subA.sum(axis=1)) - subA
    eigvals = np.sort(eigvalsh(L))

    if method == 'eigenValue2':
        # Return the 2nd smallest eigenvalue (Fiedler value)
        return eigvals[1] if len(eigvals) > 1 else 0.0
    elif method == 'eigenValue3':
        # Return the 3rd smallest eigenvalue
        return eigvals[2] if len(eigvals) > 2 else 0.0
    elif method == 'nonZeroEigen':
        # Return the smallest positive eigenvalue
        pos_eigvals = eigvals[eigvals > 1e-8]
        return pos_eigvals[0] if len(pos_eigvals) > 0 else 0.0
    else:
        raise NotImplementedError(f"Feature method '{method}' not implemented.")

def ff_imputation_lr_model(adj_matrix, core_indices, fringe_indices, method='eigenValue2', threshold=0.1, lr_kwargs=None):
    """
    Impute the FF block using a LR model trained on core-core and core-fringe edges, by using the feature determined by the 
    "method" param.
    """
    if lr_kwargs is None:
        lr_kwargs = {'solver': 'liblinear', 'max_iter': 1000}

    A = adj_matrix.toarray() if hasattr(adj_matrix, 'toarray') else np.array(adj_matrix)
    n = A.shape[0]
    A_pred = A.copy()
    added_edges = 0
    L = len(fringe_indices)
    # build the training set 
    X_train = []
    y_train = []

    for u in core_indices:
        for v in core_indices:
            if u >= v:
                continue
            feature = extract_feature(A, u, v, method)
            X_train.append([feature])
            y_train.append(A[u,v])

    for u in core_indices:
        for v in fringe_indices:
            feature = extract_feature(A, u, v, method)
            X_train.append([feature])
            y_train.append(A[u,v])

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    # train LR model 
    model = LogisticRegression(**lr_kwargs)
    model.fit(X_train, y_train)

    # predict fringe-fringe edges
    for u in fringe_indices:
        for v in fringe_indices:
            if u >= v:
                continue 

            feature = extract_feature(A, u, v, method)
            prob = model.predict_proba([[feature]])[0,1]
            if prob >= threshold:
                A_pred[u,v] = 1
                A_pred[v,u] = 1
                added_edges += 1

    print(f"Possible FF Edges: {L * (L-1)} \t Added Edges : {added_edges * 2}")
    return A_pred, added_edges



    
    

In [10]:
def auc_confidence_interval(y_true, y_scores, n_bootstraps=1000, random_seed=42):
    rng = np.random.RandomState(random_seed)
    bootstrapped_scores = []
    for i in range(n_bootstraps):
        indices = rng.randint(0, len(y_true), len(y_true))
        if len(np.unique(y_true[indices])) < 2:
            # skip if only one class in the sample
            continue
        score = roc_auc_score(y_true[indices], y_scores[indices])
        bootstrapped_scores.append(score)
    sorted_scores = np.sort(bootstrapped_scores)
    lower = sorted_scores[int(0.025 * len(sorted_scores))]
    upper = sorted_scores[int(0.975 * len(sorted_scores))]
    return lower, upper
    
def logistic_regression_model(
    adj_matrix, core_indices, 
    fringe_indices, metadata, 
    feature='link', lr_kwargs=None, 
    seed=None, threshold=0.5,
    ff_imputation='eigenValue2'
):
    gender = metadata[:, 1].astype(int)  # Convert to integer
    dorm = metadata[:, 4]
    added_edges = 0
    if ff_imputation:
        # impute the fringe-fringe edges 
        ff_imputed_adj_matrix, added_edges = ff_imputation_lr_model(adj_matrix, core_indices, fringe_indices, method=ff_imputation, threshold=threshold, lr_kwargs=lr_kwargs)
    else:
        ff_imputed_adj_matrix = adj_matrix
    if feature == 'link':
        X_train = ff_imputed_adj_matrix[core_indices, :]
        y_train = gender[core_indices]
        X_test = ff_imputed_adj_matrix[fringe_indices, :]
    elif feature == 'triangles':
        # @toDo: implement this
        X_train = ff_imputed_adj_matrix[core_indices, :]
        y_train = gender[core_indices]
        X_test = ff_imputed_adj_matrix[fringe_indices, :]
    elif feature == 'node2vec':
        # @toDo: implement this
        X_train = ff_imputed_adj_matrix[core_indices, :]
        y_train = gender[core_indices]
        X_test = ff_imputed_adj_matrix[fringe_indices, :]
        
    print("\n Feature Space (Core-Fringe)")
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    y_test = gender[fringe_indices]
    unique_train_classes = np.unique(y_train)
    print(f"Unique training classes: {unique_train_classes}")
    model = LogisticRegression(**lr_kwargs, random_state=seed)
    model.fit(X_train, y_train)
    beta = model.coef_.flatten()
    # print(f"\nModel Analysis:")
    # print(f"Number of non-zero coefficients: {np.count_nonzero(beta)}")
    # print(f"Mean absolute coefficient: {np.mean(np.abs(beta)):.4f}")
    # print(f"Max coefficient: {np.max(np.abs(beta)):.4f}")
    # print(f"Min coefficient: {np.min(np.abs(beta)):.4f}")
    # print(f"Max coefficient (No-Abs): {np.max(beta):.4f}")
    # print(f"Min coefficient (No-Abs): {np.min(beta):.4f}")

    y_test_pred = model.predict(X_test)
    y_test_scores = model.predict_proba(X_test)
    
    # Verify class order and AUC calculation
    print("\nClass Order Verification:")
    print(f"Model classes_: {model.classes_}")  # Order of classes in the model
    print(f"Unique test classes: {np.unique(y_test)}")  # Classes in test set
    print(f"Class distribution in test: {dict(Counter(y_test))}")
    print(f"Prediction distribution: {dict(Counter(y_test_pred))}")

    # Calculate AUC for each class
    for i, class_label in enumerate(model.classes_):
        class_auc = roc_auc_score(y_test == class_label, y_test_scores[:, i])
        print(f"AUC for class {class_label}: {class_auc:.4f}")
    
    # Use the correct class index for AUC
    positive_class_idx = np.where(model.classes_ == 2)[0][0]  # Assuming 2 is our positive class
    auc = roc_auc_score(y_test, y_test_scores[:, positive_class_idx])
    accuracy = accuracy_score(y_test, y_test_pred)
    
    # Compute AUC confidence interval
    auc_lower, auc_upper = auc_confidence_interval(y_test, y_test_scores[:, positive_class_idx])
    print(f"AUC 95% CI: [{auc_lower:.3f}, {auc_upper:.3f}]")
    
    print(f"\nFinal Results:")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test ROC AUC: {auc:.4f}")
    print(f"Training class distribution: {dict(Counter(y_train))}")
    print(f"Test class distribution: {dict(Counter(y_test))}")

    return beta, accuracy, auc, (auc_lower, auc_upper), added_edges
    

In [11]:
seed = random.seed(datetime.now().timestamp())
sbm_adj_matrix, metadata = sbm_gender_homophily_adj_and_metadata(500, 500, 0.15, 0.1, seed=seed)
print(sbm_adj_matrix.shape)
print(metadata.shape)
print(metadata[:, 1].shape)
adj_matrix, core_indices, fringe_indices, ff_true = create_iid_core_fringe_graph(sbm_adj_matrix, 300, seed=seed, ff=True)
print(adj_matrix.shape)
print(len(core_indices))
print(len(fringe_indices))
lr_kwargs = {'C': 100, 'solver': 'liblinear', 'max_iter': 1000}
_, acc, auc, auc_ci, added_edges = logistic_regression_model(adj_matrix, core_indices, fringe_indices, 
                                                metadata, feature='link', lr_kwargs=lr_kwargs, 
                                                seed=seed, threshold=0.10, ff_imputation='eigenValue2')
L = len(fringe_indices)
print(f"Possible FF Edges: {L * (L-1)} \t Added Edges : {added_edges * 2}")

(1000, 1000)
(1000, 7)
(1000,)
Total edges in original adjacency: 62779
IID core
Core size: 300
Number of core-core edges: 5632
Number of core-fringe edges: 13129
Number of fringe-fringe edges (lost): 44018
(1000, 1000)
300
700


KeyboardInterrupt: 